# Node Set Extraction from existing npz file.

Since projections can be quite big, use bigmem or it will go OOM.
Future implementations will use dask.

In [10]:
from scipy import sparse
import numpy as np
from bluepysnap import Circuit

In [8]:
c = Circuit('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/sonata/circuit_config.json')
nodes = c.nodes['hippocampus_neurons']

In [11]:
class NodeSetExtractor:
    def __init__(self,circuit,adjacency_matrix,population='hippocampus_neurons'):
        self.adjacency_matrix = adjacency_matrix
        self.nodes = circuit.nodes[population]


    def extract_from_local(self,node_set):
        '''
        Parameters
        ----------
        full_adjacency : scipy.sparse.csr_matrix
            Full adjacency matrix
        node_set : list
            node set name for the target adj to be extracted
        '''
        node_indices = self.nodes.ids(node_set)
        return self.adjacency_matrix[node_indices,:][:,node_indices]
    
    def extract_with_projections(self,node_set,projection_matrix_path:str):
        projection_matrix = sparse.load_npz(projection_matrix_path)
        node_indices = self.nodes.ids(node_set)
        local_target_adj = self.adjacency_matrix[node_indices,:][:,node_indices] # 6k x 6k for cylinder 300
        return np.concatenate([local_target_adj,projection_matrix[:,node_indices]],axis=0) # 6k+N x 6k for cylinder 300
        

In [16]:
local_matrix_path = '/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/data/ca1_synaptome.npz'
projection_matrix_path = '/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/data/ca3_synaptome.npz'

In [17]:
local_synaptome = sparse.load_npz(local_matrix_path)

In [18]:
extractor = NodeSetExtractor(c,local_synaptome)

In [19]:
target_adj_wproj = extractor.extract_with_projections('slice39',projection_matrix_path)